# 策略说明

如果股价沿趋势线上升，那么当其回到支撑线或者RSI处于低位时，可以买入

In [2]:
await init_notebook()
def feature_trendline(code, name, bars, results, ft):
    alt = altitude(bars)
    mf = MaLineFeatures()
    
    vec = mf.feature(bars[:-1], lines=6)
    f = mf.to_dict(vec)
    
    close = bars["close"]
    roc = close[-2:]/close[-3:-1]-1
    if f["trendline_slope"] > 0 and np.all(roc > 0):
        results.append((name, code, alt, f["trendline_slope"], f["trendline"], f["trendline_gap"], bars["close"][-1]))

NameError: name 'init_notebook' is not defined

In [ ]:
get_ipython().profile_dir.startup_dir

In [ ]:
def feature_parallel(code, name, bars, results, ft):
    alt = altitude(bars)
    mf = MaLineFeatures()
    
    vec = mf.feature(bars, lines=4)
    f = mf.to_dict(vec)
    
    if f["parallel"] > 0:
        results.append((name, code, alt, f["parallel"], bars[-1]["close"]))

In [ ]:
results = await scan(feature_trendline, 139, nstocks=-1, silent=True)
df = pd.DataFrame(results, columns=["naem", "code", "alt", "slope", "trendline","gap", "price"])

In [ ]:
df[(df.trendline==5)&(df.gap<0.05)].nlargest(10, "slope")

In [ ]:
results = await scan(feature_parallel, 60, nstocks=-1, silent=True)
df = pd.DataFrame(results, columns=["naem", "code", "alt", "parallel", "price"])

In [ ]:
df[(df.price<5)&(df.parallel<5)&(df.parallel>2)]

In [ ]:
results = await scan(feature_cross, 129, nstocks=-1, silent=True)
df = pd.DataFrame(results, columns=["name", "code", "alt", "slope", "line", "price"])

# 均线黄金坑

In [ ]:
def ma(code, name, bars, results, ft):
    alt = altitude(bars)
    mf = MaLineFeatures()
    
    vec = mf.feature(bars, lines=5)
    results.append((code, name, *vec))

results = await scan(ma, 80, tm="2021-10-13", nstocks=-1)


In [ ]:
cols = [col[0] for col in MaLineFeatures().columns_60]
df = pd.DataFrame(results, columns=["code", "name", *cols])
show_cols = ["name","trendline", "trendline_slope", "a5", "b5", "vx5", "a10", "b10", "vx10", "parallel_234"]
df[(df.trendline!=-1) & (df.trendline_slope>0) &(df.a5>5e-3) & (df.b5<-1e-2)&(df.parallel_234<21)][show_cols]

# 周均线支撑

In [ ]:
bars = jq.get_bars("000875.XSHE", 100, '1w', end_dt='2021-12-03',df=False, include_now=True)
print(bars["close"])

In [ ]:
#def week_20_features(code, name, bars, results, ft):
bars = await get_bars("000875.XSHE", 100, '1w', '2021-12-03')
print(np.round(bars["close"],2)[19:])
print(bars["frame"][19:])
ma = moving_average(bars["close"], 20)
print(np.round(ma,2))

# align
bars = bars[19:]

close = bars["close"]
low = bars["close"]
frame = bars["frame"]

cross_points = np.argwhere((low*0.98 < ma) & (close > ma)).flatten()
print(frame[cross_points])

In [ ]:
bars_om = await get_bars("000875.XSHE", 24, '1w', '2021-12-03')
print(len(bars_om), bars_om["close"])
print(bars_om["frame"])

In [ ]:
bars_jq = jq.get_bars("000875.XSHE", 24, '1w', end_dt='2021-12-03 15:00:00', include_now=True, df=False)
print(bars_jq["date"])
np.round(moving_average(bars_jq["close"],20),2)

In [ ]:
standard = np.array([6.19, 6.29, 6.40, 6.51])

In [ ]:
standard/ma[-4:]

# 30分钟支撑线

使用120支撑线，当RSI低位时买入

In [ ]:
def feature_120_rsi(code, name, bars, results, ft):
    if len(bars) < 130:
        return
    
    close = bars["close"]
    
    isup = close[-1]/close[-2] - 1 > 0
    
    if not isup:
        return
    
    rsi = relative_strength_index(close, 6)[-3:]
    prsi = np.array([rsi30.get_proba(code, r) for r in rsi])
    
    if (prsi[0] and any(prsi < 0.1)):
        ma = moving_average(close, 120)[-7:]
        
        (a,b,_), pmae = polyfit(ma/ma[0])

        if abs(a) < 3e-3 and b > 5e-4 and pmae < 3e-4:
            results.append((code, name, bars[-1]["frame"], b, min(rsi), min(prsi)))
    elif any(rsi < 10):
        ma = moving_average(close, 120)[-7:]
        
        (a,b,_), pmae = polyfit(ma/ma[0])
        if abs(a) < 3e-3 and b > 5e-4 and pmae < 3e-4:
            results.append((code, name, bars[-1]["frame"], b, min(rsi), None))

In [ ]:
results = await scan(feature_120_rsi, 140, '30m', nstocks=-1, tm="2021-12-06 10:00")

In [ ]:
df = pd.DataFrame(results, columns=["code", "name", "frame", "b", "rsi", "prsi"])
df

In [3]:
from alpha.notebook import init_notebook

ModuleNotFoundError: No module named 'talib'